## Noise

- Measure noise from continuum images

We measure the noise of images from 24 frequency intervals, between 84 MHz and 217 GHz.

In [18]:
import numpy as np
import shutil
from astropy.io import ascii, fits
import fluxInt
import pbCorr
import synCl
from prettytable import PrettyTable
from astropy.table import Table

fint = fluxInt.flInt()
pb = pbCorr.pbcorr()
syn = synCl.synClean()

rootDir = '/home/maccagni/Projects/MFS/FornaxA/contData/'
regDir = rootDir+'inIms/regions/'
imDir = rootDir+'inIms/'
inTabs = rootDir+'inTabs/'

fint.rootdir = rootDir
syn.rootdir = rootDir
pb.rootdir = rootDir

aut = 'FgCv'

tableName = inTabs+'inputFluxTable'+aut+'.tbl'
tableFileNames= ascii.read(tableName,format='csv')

outTable = inTabs+'noiseFluxInt'+aut+'.tbl'
frList = [ 84.0,114.5,200.5, 320.0,1029.9794921,1440.5292968,1500.0,
      5700.0,5900.0,6100.0,6300.0,6500.0,6700.0,6874.0,100000.0,143000.0, 217000.0]
#frList = [ 84.0,114.5,155,200.5, 320.0,1029.9794921,1440.5292968,1500.0,
#      5700.0,5900.0,6100.0,6300.0,6500.0,6700.0,6874.0,
#          30000.0,44000.0,70000.0,100000.0,143000.0, 217000.0,353000.0,545000.0,857000.0]
print '''\t+---------+\n\t Inputs loaded\n\t+---------+'''


	+---------+
	 Inputs loaded
	+---------+


In [19]:
totReg = regDir+'ToT.reg'
elReg = regDir+'EL.reg'
wlReg = regDir+'WL.reg'

regions = [totReg, elReg, wlReg]

errPerc=  np.array(tableFileNames.columns[2],dtype=float)

fr=[]
rmsArray= np.zeros([len(regions),len(tableFileNames.columns[0])])
noiseArray= np.zeros([len(regions),len(tableFileNames.columns[0])])

cutoff= np.zeros([len(tableFileNames.columns[0])])

for i in xrange(0,15):

    fileName = imDir+tableFileNames.columns[0][i]

    datas,heads = fint.openFile(fileName)
    datas=np.squeeze(datas)
    heads=fint.cleanHead(heads)

    if not 'CDELT2' in heads:
        heads['CDELT2'] = heads['CD2_2']
        heads['CDELT1'] = heads['CD1_1']
    #if 'Frequency' in tableFileNames.dtype.names:
    #    freq = float(tableFileNames.columns[1][i])*1e6
    #    fr.append(freq)
    #elif 'FREQ' in heads:
    #    freq = float(heads['FREQ'])
    #    fr.append(freq)
    #elif 'CRVAL3' in heads:
    #    freq = float(heads['CRVAL3'])
    #    fr.append(freq)
    #else:
    freq = frList[i]*1e6
    fr.append(freq)
        
    beamArea = 2*np.pi*float(heads['BMAJ'])*3600./2.35482*float(heads['BMIN'])*3600./2.35482
    pixArea = -float(heads['CDELT2']*3600.)*float(heads['CDELT1']*3600.)
    numPixBeam= beamArea/pixArea   
    
    for j in xrange (0,len(regions)):
        
        if j==0:
            background, rms, pixels = fint.noiseReg(datas,heads,regions[j])
             
        else:
            backround, rmsEL, pixels = fint.noiseReg(datas,heads,regions[j])
        
        noiseArray[j,i] = rms*pixels/numPixBeam
        
    rmsArray[:,i] = rms

for i in xrange(15,len(tableFileNames)):
    
        fileName = imDir+tableFileNames.columns[0][i]
    
        datas,heads = fint.openFile(fileName)
        datas=np.squeeze(datas)
        heads=fint.cleanHead(heads)
        #if not 'CDELT2' in heads:
        #    heads['CDELT2'] = heads['CD2_2']
        #    heads['CDELT1'] = heads['CD1_1']
        #if 'Frequency' in tableFileNames.dtype.names:
        #    freq = float(tableFileNames.columns[1][i])*1e6
        #    fr.append(freq)
        #elif 'FREQ' in heads:
        #    freq = float(heads['FREQ'])
        #    fr.append(freq)
        #elif 'CRVAL3' in heads:
        #    freq = float(heads['CRVAL3'])
        #    fr.append(freq)
        #else:
        freq = frList[i]*1e6
        fr.append(freq)
        print freq
        beamArea = 2*np.pi*float(heads['BMAJ'])*3600./2.35482*float(heads['BMIN'])*3600./2.35482
        pixArea = -float(heads['CDELT2']*3600.)*float(heads['CDELT1']*3600.)
        numPixBeam= beamArea/pixArea   
        print pixArea
        for j in xrange (0,len(regions)):
            
            if j==0:
                background, rms, pixels = fint.noiseMultiReg(datas,heads,regDir+'noiseReg/')
                background1, rms1, pixels1 = fint.noiseReg(datas,heads,regions[j])
                print rms1, rms
                rms_or = rms
                noiseArray[0,i] = rms*pixels1/numPixBeam
                print noiseArray[0,i]
    
            else:
                 backround, rmsEL, pixels = fint.noiseReg(datas,heads,regions[j])
                 noiseArray[j,i] = rms*pixels/numPixBeam
                
        rmsArray[:,i] = rms


print '''\t+---------+\n\t Noise measured\n\t+---------+'''
  

1.43e+11
3600.0
30
30.0
0.0008443716478583443 0.03613582440818606
1.4941555115962555
2.17e+11
3600.0
30
30.0
0.000656461309732984 0.03169411490857978
1.3104983005477446
	+---------+
	 Noise measured
	+---------+


**Output tables**:

```
~/Projects/MFS/FornaxA/contData/noiseFluxes.tbl'

```


In [21]:
colNames = ['#Filenames','FlCorr','ErrPerc','rmsT [Jy/pix]', 'rmsEL [Jy/pix]', 'rmsWL [Jy/pix]',
            'noiseToT [Jy]', 'noiseEL [Jy]', 'noiseWL [Jy]']

flCorr = np.array(tableFileNames.columns[1],dtype=float)
errPerc=  np.array(tableFileNames.columns[2],dtype=float)

alldata = np.array([tableFileNames.columns[0],flCorr,errPerc,rmsArray[0,:].T,rmsArray[1,:].T,rmsArray[2,:].T,
                    noiseArray[0,:].T,noiseArray[1,:].T,noiseArray[2,:].T])

tt = Table(alldata.T,names=colNames,meta={'name': 'Noise table'})

ascii.write(tt,outTable, overwrite=True)

t = PrettyTable(colNames)
t.add_row(alldata)


print t
print '''\t+---------+\n\t Tables Saved\n\t+---------+'''


+--------------------------------------------------------------------------+--------------------------------------------------------------------------+-----------------------------------------------------------------------+----------------------------------------------------------------------------+----------------------------------------------------------------------------+----------------------------------------------------------------------------+----------------------------------------------------------------------------+----------------------------------------------------------------------------+----------------------------------------------------------------------------+
|                                #Filenames                                |                                  FlCorr                                  |                                ErrPerc                                |                               rmsT [Jy/pix]                                |              